Using Content Based Filtering to recommend similar items by:

* Creating user-feature matrix
* Creating item-feature matrix
* Measuring similraity using dot product as metric
* Recommending top-k similar items

Second Approach

* Perform dimensionality reduction using PCA on user_feature and item_feature matrices

I used rows from transactions record

Limited customers to customers who bought at least two items

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [3]:
sale_path = '/content/drive/Shareddrives/Datathon/Datathon - HUIT_MEOW/VietNamDatathon-HUIT_MEOW/sales_and_inventory_mentor_data/Sales_mentor/TT T12-2022_split_2.xlsx'
products_path = '/content/drive/Shareddrives/Datathon/Datathon - HUIT_MEOW/VietNamDatathon-HUIT_MEOW/sales_and_inventory_mentor_data/MasterData/Productmaster.xlsx'

In [4]:
sale_df = pd.read_excel(sale_path)
sale_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180673 entries, 0 to 180672
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   month                      180673 non-null  int64 
 1   week                       180673 non-null  int64 
 2   site                       180673 non-null  int64 
 3   branch_id                  180673 non-null  int64 
 4   channel_id                 180673 non-null  object
 5   distribution_channel       180673 non-null  object
 6   distribution_channel_code  180673 non-null  object
 7   sold_quantity              180673 non-null  int64 
 8   cost_price                 180673 non-null  int64 
 9   net_price                  180673 non-null  int64 
 10  customer_id                180673 non-null  object
 11  product_id                 180673 non-null  object
dtypes: int64(7), object(5)
memory usage: 16.5+ MB


In [5]:
product_df = pd.read_excel(products_path)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94867 entries, 0 to 94866
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            94867 non-null  int64  
 1   index                 94867 non-null  int64  
 2   color                 94822 non-null  object 
 3   color_group           94867 non-null  object 
 4   listing_price         94840 non-null  float64
 5   price_group           94840 non-null  object 
 6   gender                94867 non-null  object 
 7   product_group         94867 non-null  object 
 8   detail_product_group  94867 non-null  object 
 9   shoe_product          94867 non-null  object 
 10  size_group            94867 non-null  object 
 11  size                  94722 non-null  float64
 12  age_group             94867 non-null  object 
 13  activity_group        94867 non-null  object 
 14  collection            94862 non-null  object 
 15  image_copyright    

In [6]:
users = sale_df
df = users.merge(product_df, on='product_id')
df.head()

,month,week,site,branch_id,channel_id,distribution_channel,distribution_channel_code,sold_quantity,cost_price_x,net_price,...,mc_description,name_description,mch_description,code_lock,option,cost_price_y,product_syle_color,product_syle,brand_name,vendor_name
0,2022012,202251,1813,1800,CHTT,Bán lẻ,FP,-1,-524095,-771000,...,GTT Cao Cấp Nam,"GTT Cao Cấp Nam, Xám, 41",NHÓM GÌAY THỂ THAO,NaN,P05-C05-L**,510000.0,f42cf69a1f82486d8ab4a575686d2ab5XAM,f42cf69a1f82486d8ab4a575686d2ab5,Brand2,vendor0
1,2022012,202252,1218,1200,CHTT,Bán lẻ,FP,1,524095,771000,...,GTT Cao Cấp Nam,"GTT Cao Cấp Nam, Xám, 41",NHÓM GÌAY THỂ THAO,NaN,P05-C05-L**,510000.0,f42cf69a1f82486d8ab4a575686d2ab5XAM,f42cf69a1f82486d8ab4a575686d2ab5,Brand2,vendor0
2,2022012,202249,1203,1200,CHTT,Bán lẻ,FP,1,524095,771000,...,GTT Cao Cấp Nam,"GTT Cao Cấp Nam, Xám, 41",NHÓM GÌAY THỂ THAO,NaN,P05-C05-L**,510000.0,f42cf69a1f82486d8ab4a575686d2ab5XAM,f42cf69a1f82486d8ab4a575686d2ab5,Brand2,vendor0
3,2022012,202251,1800,1800,ONLINE,Online,ZF2,1,524095,728337,...,GTT Cao Cấp Nam,"GTT Cao Cấp Nam, Xám, 41",NHÓM GÌAY THỂ THAO,NaN,P05-C05-L**,510000.0,f42cf69a1f82486d8ab4a575686d2ab5XAM,f42cf69a1f82486d8ab4a575686d2ab5,Brand2,vendor0
4,2022012,202250,1252,1200,CHTT,Bán lẻ,FP,1,524095,771000,...,GTT Cao Cấp Nam,"GTT Cao Cấp Nam, Xám, 41",NHÓM GÌAY THỂ THAO,NaN,P05-C05-L**,510000.0,f42cf69a1f82486d8ab4a575686d2ab5XAM,f42cf69a1f82486d8ab4a575686d2ab5,Brand2,vendor0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180664 entries, 0 to 180663
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   month                      180664 non-null  int64  
 1   week                       180664 non-null  int64  
 2   site                       180664 non-null  int64  
 3   branch_id                  180664 non-null  int64  
 4   channel_id                 180664 non-null  object 
 5   distribution_channel       180664 non-null  object 
 6   distribution_channel_code  180664 non-null  object 
 7   sold_quantity              180664 non-null  int64  
 8   cost_price_x               180664 non-null  int64  
 9   net_price                  180664 non-null  int64  
 10  customer_id                180664 non-null  object 
 11  product_id                 180664 non-null  object 
 12  Unnamed: 0                 180664 non-null  int64  
 13  index                      18

In [8]:
feature_subset = ['color_group', 'price_group', 'detail_product_group', 'size_group', 'age_group', 'activity_group', 'lifestyle_group', 'brand_name']

In [9]:
df1 = df[['customer_id', 'product_id'] + feature_subset]
dummies_df = pd.get_dummies(df1, columns=feature_subset)
dummies_df

,customer_id,product_id,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
0,6ba416cf1,f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1,306e0075c,f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2,f763fca95,f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,02757bde1,f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
4,0b2da01e9,f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180659,d6533d315,c1c730884b964707a779cd126830dd0dXAM36,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
180660,6c31dde16,ca261cf5500b47508539f44eb0bd0cabDOD32,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
180661,342cbd03f,2b8251fd4c884bb7952c183192fbd6e4XMN32,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
180662,948340f03,277dc426eba84657bdb51ac1fa5a6c69TRG33,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


In [10]:
minimum_items = 1
groupby_customer = dummies_df.groupby('customer_id')

l = []
cutomer_ids = []
article_ids = []
for key in groupby_customer.groups.keys():
    temp = groupby_customer.get_group(key)
    if temp.product_id.nunique() >= minimum_items:
        l.append(temp.drop('product_id', axis=1).sum(numeric_only=True).values)
        cutomer_ids.append(key)
        article_ids.extend(temp.product_id.values.tolist())

In [11]:
user_feature = pd.DataFrame(l, columns = dummies_df.columns[2:])
normalized_user_feature = user_feature.div(user_feature.sum(axis=1), axis=0)
normalized_user_feature.insert(0, 'customer_id', cutomer_ids)
normalized_user_feature = normalized_user_feature.set_index('customer_id')
normalized_user_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.000000,0.027322,0.052596,0.000683,0.040301,0.004098,0.0,0.000000,0.000000,0.103142,...,0.000000,0.026639,0.000000,0.000000,0.000000,0.098361,0.103142,0.021858,0.0,0.0
007e6bb72,0.000000,0.039216,0.024510,0.000000,0.056373,0.004902,0.0,0.007353,0.063725,0.046569,...,0.000000,0.083333,0.000000,0.000000,0.000000,0.041667,0.117647,0.007353,0.0,0.0
008e9d4fd,0.000000,0.015078,0.035019,0.003405,0.056420,0.015078,0.0,0.005837,0.037451,0.041829,...,0.000000,0.030642,0.000000,0.000000,0.000000,0.094358,0.084144,0.040856,0.0,0.0
00981a197,0.000000,0.016484,0.037088,0.009615,0.043956,0.017857,0.0,0.002747,0.020604,0.046703,...,0.005495,0.030220,0.000000,0.005495,0.000000,0.089286,0.075549,0.049451,0.0,0.0
01347a866,0.000000,0.000000,0.008333,0.008333,0.058333,0.050000,0.0,0.000000,0.000000,0.083333,...,0.000000,0.008333,0.000000,0.000000,0.000000,0.116667,0.066667,0.058333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe306672e,0.004386,0.009868,0.027047,0.002558,0.059942,0.021199,0.0,0.042763,0.021930,0.032164,...,0.002558,0.098319,0.000365,0.001827,0.005117,0.019371,0.115132,0.009868,0.0,0.0
fea9c0a58,0.000000,0.000000,0.023438,0.000000,0.101562,0.000000,0.0,0.042969,0.058594,0.023438,...,0.000000,0.062500,0.000000,0.000000,0.000000,0.062500,0.125000,0.000000,0.0,0.0
ff4287d0a,0.000000,0.008621,0.012931,0.000000,0.090517,0.012931,0.0,0.000000,0.017241,0.030172,...,0.000000,0.038793,0.000000,0.000000,0.000000,0.086207,0.047414,0.077586,0.0,0.0


In [12]:
item_feature = dummies_df.drop_duplicates(subset='product_id')
item_feature = item_feature[item_feature.product_id.isin(article_ids)].drop('customer_id', axis=1)
item_feature = item_feature.set_index('product_id')
item_feature

,color_group_KHÁC,color_group_NHIỀU MÀU,color_group_SÁNG,color_group_TRẮNG,color_group_TỐI,color_group_ĐEN,color_group_ĐEN,price_group_100<200,price_group_200<300,price_group_300<600,...,activity_group_Văn phòng,lifestyle_group_Casual,lifestyle_group_Fashion,lifestyle_group_Formal,lifestyle_group_Khác,lifestyle_group_Sport,brand_name_Brand1,brand_name_Brand2,brand_name_Brand3,brand_name_Brand4
product_id,,,,,,,,,,,,,,,,,,,,,
f42cf69a1f82486d8ab4a575686d2ab5XAM41,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
19531bbadb1741339dcd600bab37dfa3DEN38,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
450f8273b50d4c8e9bc324cd5798fc66DEN42,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
58cf4d7e9ffd4bfcaab29c971fc93c50DOO30,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
6fa98f1c52454ea39cc4cf35bb6ffee7NAU40,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c1c730884b964707a779cd126830dd0dXAM36,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
ca261cf5500b47508539f44eb0bd0cabDOD32,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2b8251fd4c884bb7952c183192fbd6e4XMN32,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0


In [13]:
scores = normalized_user_feature.dot(item_feature.T)
scores

product_id,f42cf69a1f82486d8ab4a575686d2ab5XAM41,19531bbadb1741339dcd600bab37dfa3DEN38,450f8273b50d4c8e9bc324cd5798fc66DEN42,58cf4d7e9ffd4bfcaab29c971fc93c50DOO30,6fa98f1c52454ea39cc4cf35bb6ffee7NAU40,ac800589b2004b17a4dcae1e703062e9XMN36,ccadbb7902e34bdfb63b518b2285eb31HOG28,1c30129f91534405b5ff161bceab9b33DEN35,5248f520697b454497985c1ec564c394XNH32,d0b571578bc042cca688ec92debbedc4NAU41,...,f0fd11ea5c3a4b9c98891b7d7a805773XDG22,3e85137357624a3caa6c009353d1dc82DOO36,0d2b229d88b34c8b993cec81e3fe8261XNH29,4db84d415a114fb89ca082ec91f35017CAM35,21ee9b6694e84d0082649c6039b58a07VDB40,c1c730884b964707a779cd126830dd0dXAM36,ca261cf5500b47508539f44eb0bd0cabDOD32,2b8251fd4c884bb7952c183192fbd6e4XMN32,277dc426eba84657bdb51ac1fa5a6c69TRG33,65f5f8ecd0f1465a9a60751388406d47DOD35
customer_id,,,,,,,,,,,,,,,,,,,,,
004c2ae83,0.430328,0.333333,0.383880,0.750000,0.420082,0.641393,0.569672,0.388661,0.551230,0.314208,...,0.103142,0.466530,0.490437,0.329235,0.202869,0.641393,0.282104,0.569672,0.762295,0.345628
007e6bb72,0.311275,0.487745,0.529412,0.534314,0.514706,0.600490,0.642157,0.490196,0.634804,0.453431,...,0.117647,0.529412,0.419118,0.443627,0.227941,0.600490,0.404412,0.642157,0.502451,0.448529
008e9d4fd,0.495136,0.401265,0.420720,0.563716,0.415856,0.514105,0.450389,0.475195,0.551070,0.358949,...,0.084144,0.362354,0.500000,0.358463,0.215953,0.514105,0.275778,0.450389,0.542315,0.374027
00981a197,0.504121,0.368132,0.383242,0.543956,0.396978,0.449176,0.390110,0.429945,0.450549,0.329670,...,0.075549,0.331044,0.468407,0.346154,0.240385,0.449176,0.263736,0.390110,0.537088,0.358516
01347a866,0.625000,0.408333,0.500000,0.600000,0.491667,0.450000,0.341667,0.516667,0.425000,0.358333,...,0.066667,0.241667,0.575000,0.358333,0.258333,0.450000,0.250000,0.341667,0.550000,0.358333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fe306672e,0.334795,0.616228,0.550804,0.406067,0.638158,0.368421,0.447368,0.548977,0.408260,0.569444,...,0.158991,0.468933,0.454313,0.615863,0.282164,0.368421,0.474050,0.447368,0.373173,0.555190
fea9c0a58,0.347656,0.578125,0.453125,0.593750,0.554688,0.554688,0.554688,0.589844,0.691406,0.511719,...,0.125000,0.531250,0.492188,0.496094,0.261719,0.554688,0.394531,0.554688,0.515625,0.457031
ff4287d0a,0.659483,0.439655,0.379310,0.534483,0.418103,0.409483,0.362069,0.452586,0.478448,0.375000,...,0.047414,0.293103,0.504310,0.310345,0.232759,0.409483,0.228448,0.362069,0.456897,0.362069


In [14]:
def get_rcmnd(customer_id, scores):
    cutomer_scores = scores.loc[customer_id]
    customer_prev_items = groupby_customer.get_group(customer_id)['product_id']
    prev_dropped = cutomer_scores.drop(customer_prev_items.values)
    ordered = prev_dropped.sort_values(ascending=False)
    return ordered, customer_prev_items

In [15]:
k = 6
customer_id = scores.index[1]
rcmnds, prev_items = get_rcmnd(customer_id, scores)

In [16]:
rcmnds[:k]

product_id
91543719bd50408ea6fe431b5bf098eeCAM31    0.676471
c073a50e64dc416c82e4d06957a95272XNH32    0.676471
9bf7a30392ff41d3b9edd5733a835b4dXNH32    0.676471
ec90b9ed757440029d113839bedb382bXDG33    0.676471
32f9b610ec0146168c2e28fd47de3a82REU28    0.676471
12b34164f1e54e58a153924607c95580XNH37    0.676471
Name: 007e6bb72, dtype: float64

In [17]:
prev_items

682       8224062044d04d8c88d38f72f7becd07HOG27
2532      a157990ba2b0453e96dde85c1ccf88d8DEN36
6422      259e7a28df67480cafc01fcea7127dcfXAM37
12024     f3b1d48e96624dff8b8e39c471b1de2bDEN37
13234     68d55d2dd75a455eb00675f686ed27e2DEN35
16785     9faf5526057a4d05b1afa2bd8f11ada1TIM32
18347     5785af5fb96a46fc9730cd109ef14c61TIM33
25604     613de5f902a24df4a44be15ac8aa767bXNH28
27672     1f22dfa184ca471dab38f422cd55ccf0HOG35
31399     d1b5dc1b0f784b24bd1b6037d8af01ceKEM39
32158     4e03e9210ee947d8838f352cd7b96fcbXNH31
35065     57631e0158df48a98e6d8e4130054e4fXAM42
44646     1214ac593676462ba61503aa7a7c5b66XNH30
51656     644b378cc2df40f891992012c27215d5DEN38
55197     56a2cb286c124409a1983c73d6493886XNH33
60174     7fa766312475402b96739b5a6c9ae46aDEN36
62964     2b2006b0aa814217bc00e9d70cf5b127DEN34
67155     1cdec1ca2cf4467187e82b484312018fTIM29
67216     2f23ba79732d42a1b40c0b9bdd85c293XAM39
74909     0abab4b8e8484854b6d03fdea7075fcaHOG27
75894     328442f4235543fe9f58c264e4e70d